In [12]:
import gymnasium as gym
import numpy as np
import random
from gymnasium import spaces
from collections import deque

In [13]:
def bfs_reachable(grid, start, targets):
    """
    Check if all target cells are reachable from start on grid (0=free, 1=obstacle).
    """
    H, W = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    queue = deque([start])
    visited[start] = True
    reached = set()
    while queue:
        i, j = queue.popleft()
        if (i, j) in targets:
            reached.add((i, j))
            if reached == set(targets):
                return True
        for di, dj in ((1,0),(-1,0),(0,1),(0,-1)):
            ni, nj = i+di, j+dj
            if 0 <= ni < H and 0 <= nj < W and not visited[ni, nj] and grid[ni, nj] == 0:
                visited[ni, nj] = True
                queue.append((ni, nj))
    return False


In [14]:
class CoverageEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, max_steps=200, seed=None):
        super().__init__()
        self.H, self.W = 8, 8
        self.max_steps = max_steps

        # seeding for reproducibility
        self.seed(seed)

        # Action: down, up, right, left
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(0.0, 1.0, shape=(5, self.H, self.W), dtype=np.float32)

        # Define a fixed shape library
        self.shape_library = [
            np.array([[1]]),                # single cell
            np.ones((1,3), dtype=int),     # horizontal bar
            np.ones((2,2), dtype=int),     # 2x2 block
            np.array([[1,1,1],             # U-shape
                      [1,0,1],
                      [1,1,1]]),
            np.array([[1,1,0],             # L-shape
                      [1,0,0]])
        ]

    def seed(self, seed=None):
        """
        Seed the environment's RNGs for reproducible layouts.
        """
        np.random.seed(seed)
        random.seed(seed)
        return [seed]

    def reset(self, *, seed=None, options=None):
        """
        Reset the environment; returns obs (shape C×H×W), info
        Channels:
         0 = free space
         1 = obstacles
         2 = agent location
         3 = target area
         4 = visited mask (all zeros at reset)
        """


        while True:
            grid = np.zeros((self.H, self.W), dtype=int)

            num_shapes = np.random.randint(1, len(self.shape_library) + 1)

            allowed = random.sample(self.shape_library, num_shapes)

            placed = np.zeros_like(grid)
            for _ in range(num_shapes):
                shape = random.choice(allowed)
                sh, sw = shape.shape
                i = np.random.randint(0, self.H - sh + 1)
                j = np.random.randint(0, self.W - sw + 1)
                if not np.any(placed[i:i+sh, j:j+sw] & shape):
                    placed[i:i+sh, j:j+sw] |= shape
            grid = placed

            ti = np.random.randint(0, self.H - 3 + 1)
            tj = np.random.randint(0, self.W - 3 + 1)
            full_block = [(ti+di, tj+dj) for di in range(3) for dj in range(3)]
            targets = [(i,j) for (i,j) in full_block if grid[i,j] == 0]
            if not targets:
                continue

            free_cells = list(zip(*np.where(grid == 0)))
            start = random.choice(free_cells)
            if bfs_reachable(grid, start, targets):
                break

        self.grid = grid
        self.targets = set(targets)
        self.agent_pos = start
        self.visited = { start }
        self.steps = 0

        return self._get_obs(), {}

    def _get_obs(self):
        C = 5
        state = np.zeros((C, self.H, self.W), dtype=np.float32)

        # channel 0: free space (grid==0)
        state[0, :, :] = (self.grid == 0).astype(np.float32)
        # channel 1: obstacles (grid!=0)
        state[1, :, :] = (self.grid != 0).astype(np.float32)
        # channel 2: agent location
        i, j = self.agent_pos
        state[2, i, j] = 1.0
        # channel 3: target area
        for (ti, tj) in self.targets:
            state[3, ti, tj] = 1.0
        for vi, vj in self.visited:
            state[4, vi, vj] = 1.0
        
        return state

    def step(self, action):
        # Define movement vectors
        # 0 = down, 1 = up, 2 = right, 3 = left
        moves = {0: (1, 0), 1: (-1, 0), 2: (0, 1), 3: (0, -1)}
        i, j = self.agent_pos
        di, dj = moves[action]
        ni, nj = i + di, j + dj

        # Default baseline reward
        reward = 0

        # Check validity and apply penalties
        if not (0 <= ni < self.H and 0 <= nj < self.W and self.grid[ni, nj] == 0):
            # Invalid action: stay in place
            self.agent_pos = (i, j) 
        else:
            # Valid move: update position
            self.agent_pos = (ni, nj)

        # Check if on a target
        if self.agent_pos not in self.visited:
            if self.agent_pos in self.targets:
                reward = 2.0   # new target
            self.visited.add(self.agent_pos)
        else:
            reward = -1

        # Step count
        self.steps += 1

        # Terminal bonus
        terminated = True
        for (i,j) in self.targets:
            if (i,j) not in self.visited:
                terminated = False
                break
            
        truncated = (self.steps >= self.max_steps)
        if terminated:
            reward += 30.0

        return self._get_obs(), reward, terminated, truncated, {}

    def render(self, mode="human"):
        disp = np.full((self.H, self.W), '.', dtype=str)
        for (i,j) in self.targets:
            disp[i,j] = 'T'
        for (i,j) in zip(*np.where(self.grid == 1)):
            disp[i,j] = '#'
        ai, aj = self.agent_pos
        disp[ai, aj] = 'A'
        print("\n".join("".join(row) for row in disp))
        print("\n")

    def close(self):
        pass


In [15]:
from collections import deque

# Map move‐vectors to gym actions
MOVES = {
    ( 1, 0): 0,  # down
    (-1, 0): 1,  # up
    ( 0, 1): 2,  # right
    ( 0,-1): 3,  # left
}

def bfs_full_coverage(env):
    """
    Runs a BFS in the state‐space (i, j, mask) where mask is a bitmask of
    which of the 9 target cells have been visited so far.
    Returns a list of gym actions that achieves full coverage in minimal steps.
    """
    H, W = env.H, env.W
    targets = list(env.targets)
    # build an index map so each target has a bit [0..8]

    target_to_bit = {t: idx for idx, t in enumerate(targets)}
    ALL_COVERED = (1 << len(targets)) - 1

    # initial state
    start_pos = env.agent_pos
    start_mask = 0
    if start_pos in target_to_bit:
        start_mask |= 1 << target_to_bit[start_pos]

    # BFS queue: (i, j, mask)
    queue = deque([ (start_pos[0], start_pos[1], start_mask) ])
    # parent pointers: state -> (prev_state, action)
    parent = { (start_pos[0], start_pos[1], start_mask): (None, None) }

    while queue:
        i, j, mask = queue.popleft()

        # have we covered all?
        if mask == ALL_COVERED:
            # reconstruct action sequence
            actions = []
            cur = (i, j, mask)
            while parent[cur][0] is not None:
                prev, act = parent[cur]
                actions.append(act)
                cur = prev
            return list(reversed(actions))

        # otherwise expand neighbors
        for (di, dj), action in MOVES.items():
            ni, nj = i+di, j+dj
            # must be in‐bounds and free
            if not (0 <= ni < H and 0 <= nj < W and env.grid[ni, nj] == 0):
                continue

            new_mask = mask
            if (ni, nj) in target_to_bit:
                new_mask |= 1 << target_to_bit[(ni, nj)]

            state = (ni, nj, new_mask)
            if state not in parent:
                parent[state] = ((i, j, mask), action)
                queue.append(state)

    # no solution
    return None

def run_bfs(env):
    obs, _ = env.reset()
    plan = bfs_full_coverage(env)
    if plan is None:
        print("No coverage path exists!")
        return

    total_reward = 0
    for a in plan:
        obs, r, done, truncated, _ = env.step(a)
        total_reward += r
        env.render()
        if done or truncated:
            break

    print("Steps:", len(plan), "Total reward:", total_reward)


In [16]:
env = CoverageEnv(seed=42)
coverage_with_bfs(env)

........
........
..TTT.#.
..TTT...
..####..
..#.#...
.##..A..
.#......


........
........
..TTT.#.
..TTT...
..####..
..#.#A..
.##.....
.#......


........
........
..TTT.#.
..TTT...
..####..
..#.#.A.
.##.....
.#......


........
........
..TTT.#.
..TTT...
..####A.
..#.#...
.##.....
.#......


........
........
..TTT.#.
..TTT.A.
..####..
..#.#...
.##.....
.#......


........
........
..TTT.#.
..TTTA..
..####..
..#.#...
.##.....
.#......


........
........
..TTT.#.
..TTA...
..####..
..#.#...
.##.....
.#......


........
........
..TTA.#.
..TTT...
..####..
..#.#...
.##.....
.#......


........
........
..TAT.#.
..TTT...
..####..
..#.#...
.##.....
.#......


........
........
..TTT.#.
..TAT...
..####..
..#.#...
.##.....
.#......


........
........
..TTT.#.
..ATT...
..####..
..#.#...
.##.....
.#......


........
........
..ATT.#.
..TTT...
..####..
..#.#...
.##.....
.#......


Total reward (BFS): 42.0
